### align `lmdeploy` forward precision with `transformers` forward precision

In [ ]:
"transformers initialize"
from transformers import Qwen2_5_VLForConditionalGeneration, Qwen2_5_VLConfig

model_path = "/models/Qwen/Qwen2.5-VL-3B-Instruct"
config = Qwen2_5_VLConfig.from_pretrained(model_path)
transformer_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(model_path,device_map="cuda:1")

In [ ]:
"lmdpeloy initialize"
import torch
from lmdeploy import PytorchEngineConfig
from lmdeploy.pytorch.config import BackendConfig, CacheConfig, ModelConfig
from lmdeploy.pytorch.configurations import AutoModelConfigBuilder
from lmdeploy.pytorch.multimodal import MultiModalTensor
from lmdeploy.pytorch.model_inputs import StepContextManager, ModelInputs, VisionModelInputs
from lmdeploy.pytorch.models.qwen2_5_vl import Qwen2_5_VLForConditionalGeneration as lmdeploy_Qwen2_5_VL
device="cuda:0"

engine_config=PytorchEngineConfig(cache_max_entry_count=0.4)

backend_config = BackendConfig(eager_mode=False,device_type='cuda')

model_config:ModelConfig = AutoModelConfigBuilder.build(
    hf_config=config, model_path=model_path, tp=1
)
model_config.k_head_dim = model_config.v_head_dim = 128
# rich.print("[ModelConfig]\n",model_config)

cache_config = CacheConfig(
    max_batches=engine_config.max_batch_size,
    block_size=engine_config.block_size,
    num_cpu_blocks=engine_config.num_cpu_blocks,
    num_gpu_blocks=engine_config.num_gpu_blocks,
    cache_max_entry_count=engine_config.cache_max_entry_count,
    max_prefill_token_num=engine_config.max_prefill_token_num,
    enable_prefix_caching=engine_config.enable_prefix_caching,
    quant_policy=engine_config.quant_policy,
    device_type=engine_config.device_type,
)
ctx_mgr = StepContextManager()
inputs = ModelInputs(
    input_ids=...,
    seq_length=torch.Tensor([0]).to(device=device,dtype=torch.int64),
    history_lengths=torch.Tensor([0]).to(device,dtype=torch.int64),
    block_offsets=torch.arange(22,device=device,dtype=torch.int64),
    is_decoding=False,
    num_ignored_history=torch.Tensor([0]).to(device=device,dtype=torch.int64),
    local_adapter_ids=None,
    vision_inputs=VisionModelInputs(
        history_lengths=torch.Tensor([0]).to(device=device,dtype=torch.int64),
        history_image_nums=None,
        history_image_token_lengths=None,
        input_embeddings=None,
        input_embedding_ranges=None,
        input_embedding_indexing=None,
        input_multimodals={
            "image":[
                MultiModalTensor(
                    data=torch.Tensor([...]),
                    encoder_len=None,
                    end=1384,
                    meta={'grid_thw':torch.Tensor}
                )
            ]
        }
    ),
    cross_length=None,
    history_cross_length=None,
    model_metas=[None]
)